In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("data-stage2.csv")
df.head()

,Unnamed: 0,category,amt,is_fraud,hour,trans_count_7d,trans_count_30d,time_diff
0,0,misc_net,4.97,0,1,0.0,0.0,0.0
1,1,grocery_pos,107.23,0,1,0.0,0.0,0.0
2,2,entertainment,220.11,0,1,0.0,0.0,0.0
3,3,gas_transport,45.00,0,1,0.0,0.0,0.0
4,4,misc_pos,41.96,0,1,0.0,0.0,0.0


In [2]:
df = df.drop(columns = ['Unnamed: 0'])
df = pd.get_dummies(df, columns = ['category'])
df.head()


,amt,is_fraud,hour,trans_count_7d,trans_count_30d,time_diff,category_entertainment,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel
0,4.97,0,1,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,107.23,0,1,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,220.11,0,1,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,45.00,0,1,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,41.96,0,1,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [3]:
df_y_removed = df.drop(columns = ['is_fraud'])

# standardization
scaler = StandardScaler()
scaled = scaler.fit_transform(df_y_removed)
df_standardized = pd.DataFrame(scaled, columns = ['amt', 'hour', 'trans_count_7d', 'trans_count_30d', 'time_diff', 'category_entertainment', 'category_food_dining', 'category_gas_transport','category_grocery_net', 'category_grocery_pos', 'category_health_fitness', 'category_home', 'category_kids_pets', 'category_misc_net', 'category_misc_pos', 'category_personal_care', 'category_shopping_net', 'category_shopping_pos', 'category_travel']) 
# add is_fraud column back into data set
df_standardized['is_fraud'] = df['is_fraud']

## Create Model

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
# import libraries
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from numpy import array 
from numpy import argmax 
from math import sqrt
import imblearn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from collections import Counter

In [7]:
model = keras.models.Sequential([keras.Input(shape=(19),)])

layer1_num_units = np.ceil(.9*19)
layer2_num_units = np.ceil(.8*layer1_num_units)
layer3_num_units = np.ceil(.6*layer2_num_units)
layer4_num_units = np.ceil(.7*layer3_num_units)

# Create hidden layers
model.add(
    layers.Dense(
        units=layer1_num_units,
        activation="relu",
    )
)

model.add(
   layers.Dense(
        units=layer2_num_units,
        activation="relu",
    )
)

model.add(
   layers.Dense(
        units=layer3_num_units,
        activation="relu",
    )
)

model.add(
   layers.Dense(
        units=layer4_num_units,
        activation="relu",
    )
)

# Single output layer
model.add(
    layers.Dense(
        units=1, 
        activation="sigmoid"
    )
)

# Compile model, tune learning rate
model.compile(
    optimizer=keras.optimizers.SGD(1e-2),
    loss=keras.losses.BinaryCrossentropy(), 
    metrics=[
        keras.metrics.BinaryAccuracy(name="accuracy"),
        keras.metrics.Precision(name="precision"),
        keras.metrics.Recall(name="recall")
    ]
)


In [21]:
# oversampling (using training set)
# can lead to overfitting
smote = SMOTE()

In [22]:
# k-fold Cross Validation

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle = True)

# get X and Y
X, Y = df_standardized.drop(columns = ['is_fraud']), df_standardized['is_fraud']

i = 0
for train_i, test_i in kf.split(X):
    start_train, stop_train = train_i[0], train_i[-1]+1
    start_test, stop_test = test_i[0], test_i[-1]+1
    
    x_train, y_train = X[start_train:stop_train], Y[start_train:stop_train],
    x_test, y_test = X[start_test:stop_test], Y[start_test:stop_test]
    x_train_smote, y_train_smote = smote.fit_resample(x_train, y_train)
    
    history = model.fit(
    x_train_smote,
    y_train_smote,
    epochs=10,
    validation_data=(x_test, y_test)
    )
    i += 1
    print("\n=====", i, "=====")
    print("Evaluate on test data")
    results = model.evaluate(x_test, y_test)
    print("loss, accuracy, precision, recall:", results, "\n\n")

Epoch 1/10
115172/115172 [==============================] - 291s 3ms/step - loss: 0.0890 - accuracy: 0.9612 - precision: 0.9613 - recall: 0.9611 - val_loss: 0.0970 - val_accuracy: 0.9583 - val_precision: 0.1085 - val_recall: 0.9704
Epoch 2/10
115172/115172 [==============================] - 256s 2ms/step - loss: 0.0738 - accuracy: 0.9690 - precision: 0.9679 - recall: 0.9702 - val_loss: 0.0963 - val_accuracy: 0.9585 - val_precision: 0.1096 - val_recall: 0.9759
Epoch 3/10
115172/115172 [==============================] - 224s 2ms/step - loss: 0.0684 - accuracy: 0.9718 - precision: 0.9695 - recall: 0.9742 - val_loss: 0.0675 - val_accuracy: 0.9719 - val_precision: 0.1525 - val_recall: 0.9645
Epoch 4/10
115172/115172 [==============================] - 253s 2ms/step - loss: 0.0656 - accuracy: 0.9732 - precision: 0.9701 - recall: 0.9766 - val_loss: 0.0574 - val_accuracy: 0.9773 - val_precision: 0.1823 - val_recall: 0.9604
Epoch 5/10
115172/115172 [==============================] - 253s 2ms/ste

115172/115172 [==============================] - 245s 2ms/step - loss: 0.0551 - accuracy: 0.9781 - precision: 0.9743 - recall: 0.9821 - val_loss: 0.0680 - val_accuracy: 0.9702 - val_precision: 0.1467 - val_recall: 0.9789ss: 0.0550 - accuracy: 0.9781
Epoch 2/10
115172/115172 [==============================] - 243s 2ms/step - loss: 0.0550 - accuracy: 0.9781 - precision: 0.9743 - recall: 0.9822 - val_loss: 0.0514 - val_accuracy: 0.9797 - val_precision: 0.2002 - val_recall: 0.9697
Epoch 3/10
115172/115172 [==============================] - 248s 2ms/step - loss: 0.0549 - accuracy: 0.9782 - precision: 0.9745 - recall: 0.9822 - val_loss: 0.0622 - val_accuracy: 0.9742 - val_precision: 0.1656 - val_recall: 0.9784
Epoch 4/10
115172/115172 [==============================] - 246s 2ms/step - loss: 0.0547 - accuracy: 0.9783 - precision: 0.9744 - recall: 0.9823 - val_loss: 0.0774 - val_accuracy: 0.9668 - val_precision: 0.1335 - val_recall: 0.9802
Epoch 5/10
115172/115172 [============================